## Compiler Optimization

In [1]:
import torch
import os
import sys
base_dir = os.environ.get('TORCHSIM_DIR', default='/workspace/PyTorchSim')
sys.path.append(base_dir)
os.environ['TORCHSIM_FUNCTIONAL_MODE']="0"
os.environ['TORCHSIM_TIMING_MODE']="1"

### GeMM + ReLU fusion

In [2]:
from Scheduler.scheduler import PyTorchSimRunner
device = PyTorchSimRunner.setup_device().custom_device()

input = torch.randn(1024, 1024).to(device=device)
weight = torch.randn(1024, 1024).to(device=device)

def gemm_relu(a, b):
    return torch.relu(torch.matmul(a, b))

opt_fn = torch.compile(dynamic=False)(gemm_relu)
out = opt_fn(input, weight)

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/npu/build.ninja...
Building extension module npu...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module npu...


ninja: no work to do.
Wrapper Codegen Path = /tmp/torchinductor_root/3z/c3zx4dfsx2o24goyevxgy4upevdsyxegbadiquz7z33ttsbs22a6.py
[Gem5] Gem5 is running... 
[TOGSim] TOGSim is running.   
[TOGSim] Simulation of "/root/workspace/PyTorchSim/tmp/5o2xythi5z3/tile_graph.onnx" is stored to "/root/workspace/PyTorchSim/tmp/5o2xythi5z3/togsim_result/0"


In [3]:
!cat /root/workspace/PyTorchSim/tmp/5o2xythi5z3/togsim_result/0 | grep "Total execution cycle"

[2025-12-01 16:03:17.222] [info] Total execution cycles: 51715


### Disable fusion

In [4]:
os.environ['TORCHSIM_COMPILER_OPTIMIZATION']="none"

input = torch.randn(1024, 1024).to(device=device)
weight = torch.randn(1024, 1024).to(device=device)

def gemm_relu(a, b):
    return torch.relu(torch.matmul(a, b))

opt_fn = torch.compile(dynamic=False)(gemm_relu)
out = opt_fn(input, weight)

Wrapper Codegen Path = /tmp/torchinductor_root/qi/cqi2yw3qrkfpe7v2zwm2hydottmvrhqqzqmmhaen7ozcvkkcrvro.py
[Gem5] Gem5 is running... 
[Gem5] Gem5 is running..  
[TOGSim] TOGSim is running.   
[TOGSim] Simulation of "/root/workspace/PyTorchSim/tmp/x2ueokr7kg3/tile_graph.onnx" is stored to "/root/workspace/PyTorchSim/tmp/x2ueokr7kg3/togsim_result/0"
[TOGSim] TOGSim is running... 
[TOGSim] Simulation of "/root/workspace/PyTorchSim/tmp/37dfo4nczcq/tile_graph.onnx" is stored to "/root/workspace/PyTorchSim/tmp/37dfo4nczcq/togsim_result/0"


In [6]:
!cat /root/workspace/PyTorchSim/tmp/x2ueokr7kg3/togsim_result/0 | grep "Total execution cycle"
!cat /root/workspace/PyTorchSim/tmp/37dfo4nczcq/togsim_result/0 | grep "Total execution cycle"

[2025-12-01 16:03:27.953] [info] Total execution cycles: 49999
[2025-12-01 16:03:29.117] [info] Total execution cycles: 66093
